In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

In [5]:
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

In [ ]:
features_matrix = X
target_vector = y

print("Размер матрицы признаков:", features_matrix.shape)
print("Размер вектора целевых переменных:", target_vector.shape)

print("\nЗанимаемая память:")
print(f"Матрица признаков: {features_matrix.nbytes}")
print(f"Вектор целевых переменных: {target_vector.nbytes} байт")

Размер матрицы признаков: (150, 4)
Размер вектора целевых переменных: (150,)

Занимаемая память:
Матрица признаков: 4800
Вектор целевых переменных: 1200 байт


In [7]:
def normalize_data(data):
    min_vals = data.min(axis=0)
    max_vals = data.max(axis=0)
    normalized = (data - min_vals) / (max_vals - min_vals)
    return normalized

X_normalized = normalize_data(features_matrix)

print("До нормализации (первые пять строк):")
print(features_matrix[:5])
print("После нормализации:")
print(X_normalized[:5])

До нормализации (первые пять строк):
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
После нормализации:
[[0.22222222 0.625      0.06779661 0.04166667]
 [0.16666667 0.41666667 0.06779661 0.04166667]
 [0.11111111 0.5        0.05084746 0.04166667]
 [0.08333333 0.45833333 0.08474576 0.04166667]
 [0.19444444 0.66666667 0.06779661 0.04166667]]


In [11]:
sepal_length = X_normalized[:, 0]

def create_categories(data):
    small_threshold = np.quantile(data, 0.25)
    large_threshold = np.quantile(data, 0.75)

    categories = []
    for value in data:
        if value < small_threshold:
            categories.append('small')
        elif value > large_threshold:
            categories.append('big')
        else:
            categories.append('medium')
    
    return categories

sepal_categories = create_categories(sepal_length)

print(f"\nРаспределение категорий:")
unique, counts = np.unique(sepal_categories, return_counts=True)
for category, count in zip(unique, counts):
    print(f"{category}: {count} объекта(ов)")


Распределение категорий:
big: 35 объекта(ов)
medium: 83 объекта(ов)
small: 32 объекта(ов)


In [ ]:
def split_dataset(X, y, test_size=0.2, random_seed=14):
    np.random.seed(random_seed)
    n_samples = X.shape[0]
    n_test = int(n_samples * test_size)

    indices = np.random.permutation(n_samples)

    test_indices = indices[:n_test]
    train_indices = indices[n_test:]

    X_train = X[train_indices]
    X_test = X[test_indices]
    y_train = y[train_indices]
    y_test = y[test_indices]

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_dataset(X_normalized, target_vector)

print("Размеры выборок после разделения:")
print(f"Тренировочная выборка: {X_train.shape}")
print(f"Тестовая выборка: {X_test.shape}")
print(f"Тренировочные метки: {y_train.shape}")
print(f"Тестовые метки: {y_test.shape}")

Размеры выборок после разделения:
Тренировочная выборка: (120, 4)
Тестовая выборка: (30, 4)
Тренировочные метки: (120,)
Тестовые метки: (30,)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

svc_model1 = SVC(random_state=14)
svc_model1.fit(X_train, y_train)

y_pred1 = svc_model1.predict(X_test)
accuracy1 = accuracy_score(y_test, y_pred1)

print(f"Точность: {accuracy1}")
print("\nОтчет по классификации:")
print(classification_report(y_test, y_pred1, target_names=target_names))

Точность: 1.0

Отчет по классификации:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [16]:
svc_model2 = SVC(kernel='linear', C=1.0, random_state=14)
svc_model2.fit(X_train, y_train)
y_pred2 = svc_model2.predict(X_test)
accuracy2 = accuracy_score(y_test, y_pred2)

print(f"Точность: {accuracy2:.4f}")

print("\nОтчет по классификации:")
print(classification_report(y_test, y_pred2, target_names=target_names))

Точность: 1.0000

Отчет по классификации:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [17]:
X_train_orig, X_test_orig, y_train_orig, y_test_orig = split_dataset(
    features_matrix, target_vector
)

svc_model3 = SVC(random_state=14)
svc_model3.fit(X_train_orig, y_train_orig)
y_pred3 = svc_model3.predict(X_test_orig)
accuracy3 = accuracy_score(y_test_orig, y_pred3)

print(f"Точность: {accuracy3:.4f}")

print("\nОтчет по классификации:")
print(classification_report(y_test_orig, y_pred3, target_names=target_names))

Точность: 1.0000

Отчет по классификации:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [ ]:
import time

def evaluate_model(model : SVC, X_train, X_test, y_train, y_test, model_name):
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time

    start_time = time.time()
    y_pred = model.predict(X_test)
    predict_time = time.time() - start_time

    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name}:")
    print(f"- Точность: {accuracy:.4f}")
    print(f"- Время обучения: {train_time:.4f} сек")
    print(f"- Время предсказания: {predict_time:.4f} сек")

models = [
    (SVC(kernel='linear', C=0.01, tol=1e-2, random_state=14), "SVC simple"),
    (SVC(kernel='rbf', C=100.0, gamma=10.0, tol=1e-5, random_state=14), "SVC precise"),
    (SVC(kernel='rbf', C=1.0, gamma='scale', tol=1e-3, random_state=14), "SVC balanced")
]

for model in models:
    